In [1]:
import numpy as np
import pandas as pd
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import os

import torch
import torch.nn.functional as F
import torchvision.transforms.v2 as transforms
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader

from PIL import Image

from torchinfo import summary

torch.manual_seed(123)
np.random.seed(123)

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

torch.set_default_device(device)
print(f"Using {device}")

Using cpu


In [2]:
from bokeh.io import show, output_notebook
from bokeh.models  import TabPanel, Tabs, LegendItem, Legend, ColumnDataSource
from bokeh.plotting import figure, Row
from bokeh.layouts import column
from bokeh.transform import dodge

output_notebook()

Loading BokehJS ...

In [3]:
# Hyper Parameters

EPOCHS = 30
BATCH = 32
TEST_SPLIT = 0.2

HEIGHT = 256
WIDTH  = 256
CLASSES = ["daisy", "dandelion", "rose", "sunflower", "tulip"]

TARGET_SIZE = 5            # daisy / dandelion / rose / sunflower / tulip

DATAPATH = "flowers"

# Data Exploration and Preparation

The data utilized for this project is available [here](https://www.kaggle.com/datasets/imsparsh/flowers-dataset).

Expected data directory structure:

./flowers/ \
  ├── daisy/\
  ├── dandelion/\
  ├── rose/\
  ├── sunflower/\
  └── tulip/

Regarding sample size and class distribution, there are five flower categories: Daisy account for 764 samples (18%), 
Dandelion for 1052 (24%), Rose for 784 (18%), Sunflower for 733 (17%), and Tulip for 984 (23%), totaling 4317 samples. 
While not perfectly balanced, it is not unbalanced enough to justify over or undersampling to adjust for the bias.

In [4]:
# Data augmentation and normalization for training and validation
data_transforms = transforms.Compose([
    transforms.Resize(size=(HEIGHT,WIDTH)),
    transforms.RandomHorizontalFlip(),
    transforms.ToImage(),
    transforms.ToDtype(torch.float32, scale=True),
    transforms.Normalize([0.5, 0.5, 0.5], [0.25, 0.25, 0.25])
])

In [5]:
# Function to split dataset into train and validation sets
def split_dataset(data_dir, val_split=0.2):
    all_data = []
    for class_name in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, class_name)
        if os.path.isdir(class_dir) and not class_name.startswith('.'):
            all_data += [(os.path.join(class_dir, img), class_name) for img in os.listdir(class_dir) if not img.startswith('.')]

    train_data, val_data = train_test_split(all_data, test_size=val_split, stratify=[item[1] for item in all_data])
    return train_data, val_data

In [6]:
# Custom Dataset class
class CustomDataset(TensorDataset):
    def __init__(self, data, class_to_idx, transform=None):
        self.data = data
        self.class_to_idx = class_to_idx
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.class_to_idx[class_name]
        if self.transform:
            image = self.transform(image)
        return image, label

In [7]:
# Directory containing all images
data_dir = 'flowers'  # Replace with your dataset path

In [8]:
# Split dataset
train_data, val_data = split_dataset(data_dir, val_split=0.2)

# Get class names and indices
class_names = sorted({item[1] for item in train_data + val_data})
class_to_idx = {class_name: idx for idx, class_name in enumerate(class_names)}

# Create train and validation datasets
train_dataset = CustomDataset(train_data, class_to_idx, transform=data_transforms)
val_dataset = CustomDataset(val_data, class_to_idx, transform=data_transforms)

# Convert to torch tensors
training_loader = DataLoader(train_dataset, batch_size=BATCH, shuffle=True)
validation_loader = DataLoader(val_dataset, batch_size=BATCH, shuffle=False)

# Models Implementation

### Multiple Layer Perceptron

In [9]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(3 * HEIGHT * WIDTH, 64)
        self.fc2 = nn.Linear(64, TARGET_SIZE)

    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [10]:
summary(MLP())

Layer (type:depth-idx)                   Param #
MLP                                      --
├─Flatten: 1-1                           --
├─Linear: 1-2                            12,582,976
├─Linear: 1-3                            325
Total params: 12,583,301
Trainable params: 12,583,301
Non-trainable params: 0

### Convolutional Neural Network

In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 4, kernel_size=8, stride=1, padding=0)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(4, 16, kernel_size=4, stride=1, padding=0)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(57600, 128)
        self.fc2 = nn.Linear(128, TARGET_SIZE)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [12]:
summary(CNN())

Layer (type:depth-idx)                   Param #
CNN                                      --
├─Conv2d: 1-1                            772
├─ReLU: 1-2                              --
├─MaxPool2d: 1-3                         --
├─Conv2d: 1-4                            1,040
├─Flatten: 1-5                           --
├─Linear: 1-6                            7,372,928
├─Linear: 1-7                            645
Total params: 7,375,385
Trainable params: 7,375,385
Non-trainable params: 0

### Long-Short Term Memory

In [13]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()        
        # Bidirectional LSTM layers
        self.lstm = nn.LSTM(input_size=3 * HEIGHT, 
                            hidden_size=512,
                            num_layers=4,
                            batch_first=True,
                            bidirectional=True)
        
        self.fc1 = nn.Linear(2*512, TARGET_SIZE)

    def forward(self, x):
        x = x.permute(0, 2, 1, 3)  # (batch, 256, 3, 256)
        x = x.reshape(-1, 256, 3*256)  # (batch, 256, 3*256)
        x, (h_n, c_n) = self.lstm(x) 
        x = x[:,-1,:]
        return self.fc1(x)

In [14]:
summary(LSTM())

Layer (type:depth-idx)                   Param #
LSTM                                     --
├─LSTM: 1-1                              24,150,016
├─Linear: 1-2                            5,125
Total params: 24,155,141
Trainable params: 24,155,141
Non-trainable params: 0

### Swarm Characteristic Neural Network

In [15]:
class SwarmFilter(nn.Module):
    def __init__(self, units=32):
        super(SwarmFilter, self).__init__()
        self.units = units
        self.filter = nn.Parameter(torch.randn(units))  # Trainable parameter with random initialization

    def forward(self, x):
        # Compute the mean along the last axis, keeping the dimensions
        mean_values = torch.mean(x, dim=-1, keepdim=True)
        return mean_values * self.filter

In [16]:
class SCNN(nn.Module):
    def __init__(self):
        super(SCNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(3 * HEIGHT * WIDTH, 16)
        self.swarm1 = SwarmFilter(units=256)
        self.swarm2 = SwarmFilter(units=16)
        self.fc2 = nn.Linear(16, TARGET_SIZE) 

    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.fc1(x))  
        x = self.swarm1(x)     
        x = self.swarm2(x)     
        return self.fc2(x)

In [17]:
summary(SCNN())

Layer (type:depth-idx)                   Param #
SCNN                                     --
├─Flatten: 1-1                           --
├─Linear: 1-2                            3,145,744
├─SwarmFilter: 1-3                       256
├─SwarmFilter: 1-4                       16
├─Linear: 1-5                            85
Total params: 3,146,101
Trainable params: 3,146,101
Non-trainable params: 0

# Running the models

### Implementation

##### Visuals

In [18]:
def progressbar(i, total, start):
    x = int(100*i/total)
    print(f"\t[{ u'█' * x }{ '.' * (100-x) }] { i : 6.0f}/{ total } Waiting for {time.time()-start:5.3f} seconds", end='\r')
    if i == total:
        print()

In [19]:
def gather_stats(loss, cm, display=True):
    
    accuracy = sum([cm[i,i] for i in range(len(cm))]) / cm.sum()
    precision = []
    recall = []

    if display:
        print(f"\t{'='*102}")
        print(f"\tClassification Statistics{' '*20}Accuracy: {accuracy:.5}{' '*24}Loss: {loss:.5}")
        print(f"\t{'='*102}")
        print(f"\tClass{' '*40}Precision{' '*32}Recall")
        print(f"\t{'-'*102}")
    
    for i in range(TARGET_SIZE):
        # Extract individual confusion matrix values
        TP = cm[i, i]
        FP = cm[:, i].sum() - TP
        FN = cm[i, :].sum() - TP

        # Calculate individual metrics
        pre = TP / (TP + FP) if (TP + FP) > 0 else 0
        rec = TP / (TP + FN) if (TP + FN) > 0 else 0

        # Save the result
        precision.append(pre)
        recall.append(rec)

        if display:
            print(f"\t{CLASSES[i]:20}{' '*25}{pre:0.4f}{' '*35}{rec:0.4f}")
        
    # Calculate general metrics
    pre = sum(precision)/TARGET_SIZE
    rec = sum(recall)/TARGET_SIZE
    precision.append(pre)
    recall.append(rec)
    if display:
        print(f"\t->Macro Avg.{' '*33}{pre:0.4f}{' '*35}{rec:0.4f}")

    pre = accuracy
    rec = accuracy
    precision.append(pre)
    recall.append(rec)
    if display:
        print(f"\t->Micro Avg.{' '*33}{pre:0.4f}{' '*35}{rec:0.4f}")
        print(f"\t{'='*102}") 
    
    return accuracy, precision, recall

##### Training and Validation

In [20]:
def train_epoch(model, optimizer, criterion):
    total_loss = 0.
    confusion = np.zeros((TARGET_SIZE,TARGET_SIZE))
    size = len(training_loader)
    start = time.time()
    for i, data in enumerate(training_loader):
        # Update progress bar
        progressbar(i,size,start)
        
        # Every data instance is an input + label pair
        inputs, labels = data
        
        # Zero your gradients for every batch!
        optimizer.zero_grad()
        
        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Adjust learning weights
        optimizer.step()
        
        # Gather data and report
        preds = outputs.data.max(1)[1]
        total_loss += loss.item()
        confusion += confusion_matrix(labels.detach().numpy(), preds.detach().numpy(), labels=[0,1,2,3,4])
        
    # Finish progress bar
    progressbar(size,size,start)

    # return average loss during training
    return total_loss / size, confusion

In [21]:
def validate_epoch(model, criterion):
    total_loss = 0.
    confusion = np.zeros((TARGET_SIZE,TARGET_SIZE))
    size = len(validation_loader)
    start = time.time()
    for i, data in enumerate(validation_loader):
        # Update progress bar
        progressbar(i,size,start)
        
        # Every data instance is an input + label pair
        inputs, labels = data
        
        # Make predictions for this batch
        outputs = model(inputs)
        
        # Compute the loss and its gradients
        loss = criterion(outputs, labels)
                
        # Gather data and report
        preds = outputs.data.max(1)[1]
        total_loss += loss.item()
        confusion += confusion_matrix(labels.detach().numpy(), preds.detach().numpy(), labels=[0,1,2,3,4])

    # Finish progress bar
    progressbar(size,size,start)

    # return average loss during training
    return total_loss / size, confusion

##### Data Collection

In [22]:
def run_model(model, label="", color="navy", epochs=EPOCHS):
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    stats = {
        "Label": label,
        "Color": color,
        "Training": {
            "Loss": [],
            "Accuracy": [],
            "Precision": [],
            "Recall": [],
            "Time": 0.0,
            "Time/Epoch": 0.0
        },
        "Validation": {
            "Loss": [],
            "Accuracy": [],
            "Precision": [],
            "Recall": [],
            "Time": 0.0,
            "Time/Epoch": 0.0
        },
        "Parameters": sum(p.numel() for p in model.parameters()),
    }
    
    train_time = 0
    val_time = 0
    
    for epoch in range(epochs):
        
        # Set the model to training mode
        model.train()  
        
        # Train epoch
        print(f"Epoch {epoch + 1}/{epochs}")
        print(f"\tTraining")
        start = time.time()
        train_loss, train_cm = train_epoch(model, optimizer, criterion)
        train_time += time.time() - start
        # Print training stats for the epoch
        train_acc, train_pre, train_rec = gather_stats(train_loss, train_cm)
        
        # Run validation
        model.eval()
        val_loss = 0.0
        val_cm = np.zeros((2,2))
    
        print(f"\tValidation")
        start = time.time()
        with torch.no_grad():  # Disable gradient calculation during validation
            # Run validation
            val_loss, val_cm = validate_epoch(model, criterion)
        val_time += time.time() - start
        # Print validation stats
        val_acc, val_pre, val_rec = gather_stats(val_loss, val_cm)

        # save stats
        stats["Training"]["Loss"].append(train_loss)
        stats["Validation"]["Loss"].append(val_loss)

        # save only the final per class stats
        stats["Training"]["Accuracy"].append(train_acc)
        stats["Training"]["Precision"].append(train_pre)
        stats["Training"]["Recall"].append(train_rec)

        stats["Validation"]["Accuracy"].append(val_acc)
        stats["Validation"]["Precision"].append(val_pre)
        stats["Validation"]["Recall"].append(val_rec)
    
    stats["Training"]["Loss"] = np.array(stats["Training"]["Loss"])
    stats["Validation"]["Loss"] = np.array(stats["Validation"]["Loss"])
    
    stats["Training"]["Time"] = train_time
    stats["Training"]["Time/Epoch"] = train_time / epochs

    stats["Validation"]["Time"] = val_time
    stats["Validation"]["Time/Epoch"] = val_time / epochs
    
    return stats

### Running

In [23]:
mlp_stats = run_model(MLP(), label="MLP", color="green")

Epoch 1/30
	Training
	[████████████████████████████████████████████████████████████████████████████████████████████████████]    108/108 Waiting for 11.814 seconds
	Classification Statistics                    Accuracy: 0.29047                        Loss: 13.325
	Class                                        Precision                                Recall
	------------------------------------------------------------------------------------------------------
	daisy                                        0.2751                                   0.1931
	dandelion                                    0.3160                                   0.2090
	rose                                         0.3270                                   0.2727
	sunflower                                    0.3819                                   0.2235
	tulip                                        0.2542                                   0.5172
	->Macro Avg.                                 0.3108                 

In [24]:
cnn_stats = run_model(CNN(), label="CNN", color="navy")

Epoch 1/30
	Training
	[████████████████████████████████████████████████████████████████████████████████████████████████████]    108/108 Waiting for 17.296 seconds
	Classification Statistics                    Accuracy: 0.36693                        Loss: 1.8032
	Class                                        Precision                                Recall
	------------------------------------------------------------------------------------------------------
	daisy                                        0.3403                                   0.3470
	dandelion                                    0.3519                                   0.3967
	rose                                         0.4606                                   0.2520
	sunflower                                    0.3533                                   0.4420
	tulip                                        0.3776                                   0.3863
	->Macro Avg.                                 0.3768                 

In [25]:
lstm_stats = run_model(LSTM(), label="LSTM", color="orange")

Epoch 1/30
	Training
	[████████████████████████████████████████████████████████████████████████████████████████████████████]    108/108 Waiting for 214.414 seconds
	Classification Statistics                    Accuracy: 0.25485                        Loss: 1.5764
	Class                                        Precision                                Recall
	------------------------------------------------------------------------------------------------------
	daisy                                        0.0909                                   0.0049
	dandelion                                    0.2498                                   0.3385
	rose                                         0.2432                                   0.2265
	sunflower                                    0.3051                                   0.3191
	tulip                                        0.2431                                   0.3342
	->Macro Avg.                                 0.2264                

In [26]:
scnn_stats = run_model(SCNN(), label="SCNN", color="red")

Epoch 1/30
	Training
	[████████████████████████████████████████████████████████████████████████████████████████████████████]    108/108 Waiting for 9.393 seconds
	Classification Statistics                    Accuracy: 0.1755                        Loss: 1.5897
	Class                                        Precision                                Recall
	------------------------------------------------------------------------------------------------------
	daisy                                        0.1577                                   0.8151
	dandelion                                    0.3661                                   0.1283
	rose                                         0.0000                                   0.0000
	sunflower                                    0.0000                                   0.0000
	tulip                                        0.0000                                   0.0000
	->Macro Avg.                                 0.1048                   

# Comparative Analyse

In [27]:
models = [mlp_stats, cnn_stats, lstm_stats, scnn_stats]

### Extracting Visualization

In [28]:
# Define line plots
# Loss
loss = figure(width=800, height=400, title="Loss")
delta_loss = figure(width=800, height=400, title="Overfitting")

loss.add_layout(Legend(), 'right')
delta_loss.add_layout(Legend(), 'right')

loss.line([0], [0.5], line_width=2, legend_label="Training", line_dash="dashed",line_color="black")
delta_loss.yaxis.axis_label = "Val - Train Loss"

for fig in [loss, delta_loss]:
    fig.xaxis.axis_label = 'epoch'

# Create auxiliar arrays
x_epoches = np.arange(EPOCHS)

# Plot lines
for stats in models:
    label = stats["Label"]
    color = stats["Color"]
    line_w = 3

    # Plot loss data
    loss.line(x_epoches, stats["Validation"]["Loss"], line_width=line_w, legend_label=label,line_color=color)
    loss.line(x_epoches, stats["Training"]["Loss"], line_width=line_w, line_dash="dashed", line_color=color)

    delta = stats["Validation"]["Loss"] - stats["Training"]["Loss"]
    delta_loss.line(x_epoches, delta, line_width=line_w, legend_label=label, line_color=color)

In [29]:
# Prepare histograms
model_labels = []
model_colors = []

total_time_values = [[],[]]
epoch_time_values = [[],[]]

parameters = []

acc  = [[],[]]
pre  = [[],[]]
rec  = [[],[]]

class_labels = ["Daisy", "Dandelion", "Rose", "Sunflower", "Tulip", "Macro Avg.", "Micro Avg."]

for stats in models:
    label = stats["Label"]
    color = stats["Color"]
    model_labels.append(label)
    model_colors.append(color)

    total_time_values[0].append(stats["Training"]["Time"])
    epoch_time_values[0].append(stats["Training"]["Time/Epoch"])
    total_time_values[1].append(stats["Validation"]["Time"])
    epoch_time_values[1].append(stats["Validation"]["Time/Epoch"])


    parameters.append(stats["Parameters"])

    idx_t = -1
    idx_v = -1
    for i in range(len(stats["Training"]["Accuracy"])):
        if stats["Training"]["Accuracy"][i] > stats["Training"]["Accuracy"][idx_t]:
            idx_t = i
        if stats["Validation"]["Accuracy"][i] > stats["Validation"]["Accuracy"][idx_v]:
            idx_v = i
    
    acc[0].append(stats["Training"]["Accuracy"][idx_t])
    acc[1].append(stats["Validation"]["Accuracy"][idx_v])
    
    pre[0].append(stats["Training"]["Precision"][idx_t])
    pre[1].append(stats["Validation"]["Precision"][idx_v])
    
    rec[0].append(stats["Training"]["Recall"][idx_t])
    rec[1].append(stats["Validation"]["Recall"][idx_v])


pre = np.array(pre)
t_pre = {
    "Label": class_labels,
    "MLP":  pre[0,0],
    "CNN":  pre[0,1],
    "LSTM": pre[0,2],
    "SCNN": pre[0,3],
}
t_pre = ColumnDataSource(data=t_pre)
v_pre = {
    "Label": class_labels,
    "MLP":  pre[1,0],
    "CNN":  pre[1,1],
    "LSTM": pre[1,2],
    "SCNN": pre[1,3],
}
v_pre = ColumnDataSource(data=v_pre)

rec = np.array(rec)
t_rec = {
    "Label": class_labels,
    "MLP":  rec[0,0],
    "CNN":  rec[0,1],
    "LSTM": rec[0,2],
    "SCNN": rec[0,3],
}
t_rec = ColumnDataSource(data=t_rec)
v_rec = {
    "Label": class_labels,
    "MLP":  rec[1,0],
    "CNN":  rec[1,1],
    "LSTM": rec[1,2],
    "SCNN": rec[1,3],
}
v_rec = ColumnDataSource(data=v_rec)

In [30]:
# Time related
total_time = figure(x_range=model_labels, width=800, height=400, title="Total Running Time")
epoch_time = figure(x_range=model_labels, width=800, height=400, title="Time per Epoch")

total_time.vbar(x=model_labels, top=total_time_values[0], color="navy", width=0.75,legend_label="Training")
total_time.vbar(x=model_labels, top=total_time_values[1], color="orange", width=0.5,legend_label="Validation")

epoch_time.vbar(x=model_labels, top=epoch_time_values[0], color="navy", width=0.75,legend_label="Training")
epoch_time.vbar(x=model_labels, top=epoch_time_values[1], color="orange", width=0.5,legend_label="Validation")

for fig in [total_time, epoch_time]:
    fig.legend.location = 'top_left'
    fig.yaxis.axis_label = 'seconds'
    fig.y_range.start = 0

In [31]:
# Parameter
param = figure(x_range=model_labels, width=800, height=400, title="Parameters")
param_exp = figure(x_range=model_labels, width=800, height=400, title="Parameters", y_axis_type="log",  y_range = [10 ** 6, 10 ** 8])

param.vbar(x=model_labels, top=parameters, color="navy", width=0.75)
param_exp.vbar(x=model_labels, bottom=[10 ** 0]*len(model_labels), top=parameters, color="navy", width=0.75)

param.y_range.start = 0

In [32]:
# Metrics
t_accuracy  = figure(x_range=model_labels, width=800, height=400, title="Training Accuracy")
v_accuracy  = figure(x_range=model_labels, width=800, height=400, title="Validation Accuracy")

t_precision = figure(x_range=class_labels, width=800, height=400, title="Training Precision")
v_precision = figure(x_range=class_labels, width=800, height=400, title="Validation Precision")

t_recall    = figure(x_range=class_labels, width=800, height=400, title="Training Recall")
v_recall    = figure(x_range=class_labels, width=800, height=400, title="Validation Recall")

for fig in [
    t_accuracy, t_precision, t_recall,
    v_accuracy, v_precision, v_recall
]:
    fig.y_range.start=0
    fig.y_range.end=1
    fig.add_layout(Legend(), 'right')

t_accuracy.vbar(x=model_labels, top=acc[0], color="navy", width=0.75)
v_accuracy.vbar(x=model_labels, top=acc[1], color="navy", width=0.75)

for i, model in enumerate(model_labels):
    color = model_colors[i]

    offset = -0.27
    step = 0.18
    width = 0.15

    t_precision.vbar(x=dodge('Label', offset+i*step, range=t_precision.x_range),
                     top=model, color=color, width=width, legend_label=model, source=t_pre)
    v_precision.vbar(x=dodge('Label', offset+i*step, range=v_precision.x_range),
                     top=model, color=color, width=width, legend_label=model, source=v_pre)

    t_recall.vbar(x=dodge('Label', offset+i*step, range=t_recall.x_range),
                       top=model, color=color, width=width, legend_label=model, source=t_rec)
    v_recall.vbar(x=dodge('Label', offset+i*step, range=v_recall.x_range),
                       top=model, color=color, width=width, legend_label=model, source=v_rec)

### Visualizing

In [33]:
# Increase font size and make them all bold
for fig in [
    param, param_exp,
    loss, delta_loss,
    total_time, epoch_time,
    t_accuracy, v_accuracy,
    t_precision, v_precision,
    t_recall, v_recall,
]:
    fig.title.text_font_size = "15pt"
    fig.title.padding = 5
    
    fig.yaxis.major_label_text_font_size = "12pt"
    fig.yaxis.major_label_text_font_style = "bold"
    fig.yaxis.axis_label_text_font_size = "12pt"
    fig.yaxis.axis_label_text_font_style = "bold"
    
    fig.xaxis.major_label_text_font_size = "12pt"
    fig.xaxis.major_label_text_font_style = "bold"
    fig.xaxis.axis_label_text_font_size = "12pt"
    fig.xaxis.axis_label_text_font_style = "bold"

In [34]:
# Create tabs
tabs = Tabs(tabs=[
    TabPanel(child=column(param, param_exp), title="Size"),
    TabPanel(child=column(loss, delta_loss), title="Loss"),
    TabPanel(child=column(total_time, epoch_time), title="Time"),
    TabPanel(child=column(t_accuracy, v_accuracy), title="Accuracy"),
    TabPanel(child=column(t_precision, v_precision), title="Precision"),
    TabPanel(child=column(t_recall, v_recall), title="Recall"),
])

# Show the tabs
show(tabs)